# 6. Gibt es einen Zusammenhang dazwischen, wie man zuletzt gegen einen bestimmten Gegner gespielt hat, und wie das nächste Spiel gegen diesen Gegner ausgehen wird?

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

In [2]:
matches = pd.read_csv('../data/raw/atp_matches_till_2022.csv')

In [3]:
# Identifizierung von Spielerpaaren und deren historischen Begegnungen
# Wir erstellen ein DataFrame, das Paare von Spielern und das Ergebnis ihrer Spiele beinhaltet.

# Erstellen einer eindeutigen Identifikation für jedes Spielerpaar
matches['player_pair'] = matches.apply(lambda x: tuple(sorted([x['winner_id'], x['loser_id']])), axis=1)

# Auswählen relevanter Spalten
player_pairs_data = matches[['tourney_date', 'player_pair', 'winner_id']]

# Sortieren der Daten nach Spielerpaaren und Datum
player_pairs_data = player_pairs_data.sort_values(by=['player_pair', 'tourney_date'])

# Anzeigen der ersten paar Zeilen der neuen Tabelle
player_pairs_data


,tourney_date,player_pair,winner_id
1694,19680304,"(100001, 100039)",100001
1676,19680304,"(100001, 100073)",100073
9254,19700325,"(100001, 100074)",100074
6193,19690317,"(100001, 100119)",100119
37449,19770207,"(100001, 100324)",100324
...,...,...,...
21325,19730604,"(211289, 211298)",211298
21306,19730604,"(211296, 211300)",211296
21435,19731119,"(211306, 211311)",211306
17168,19720117,"(211306, 211376)",211376


In [4]:
# Analyse des Einflusses des letzten Spiels auf das nächste Spiel

# Berechnung des Gewinners des vorherigen Spiels für jedes Spielerpaar
player_pairs_data['prev_winner_id'] = player_pairs_data.groupby('player_pair')['winner_id'].shift(1)

# Entfernen der ersten Begegnungen, da sie kein vorheriges Spiel haben
player_pairs_data.dropna(subset=['prev_winner_id'], inplace=True)

# Umwandlung der Gewinner-ID in einen booleschen Wert: True, wenn der Gewinner des vorherigen Spiels auch das aktuelle Spiel gewonnen hat
player_pairs_data['prev_winner_won_again'] = player_pairs_data['winner_id'] == player_pairs_data['prev_winner_id']

# Berechnung der Häufigkeit, mit der der Gewinner des vorherigen Spiels auch das nächste Spiel gewinnt
prev_winner_success_rate = player_pairs_data['prev_winner_won_again'].mean()

prev_winner_success_rate

0.605718705495261

In [10]:
player_pairs_data

,tourney_date,player_pair,winner_id,prev_winner_id,prev_winner_won_again
6557,19690808,"(100002, 100005)",100002,100005.0,False
2729,19681004,"(100002, 100029)",100029,100029.0,True
6077,19690804,"(100002, 100029)",100029,100029.0,True
6552,19690808,"(100002, 100029)",100029,100029.0,True
6554,19690808,"(100002, 100030)",100002,100030.0,False
...,...,...,...,...,...
186901,20220627,"(206173, 207989)",206173,207989.0,False
187081,20220725,"(206173, 207989)",206173,206173.0,True
187461,20220829,"(206173, 207989)",207989,206173.0,False
187929,20221107,"(206909, 208103)",206909,206909.0,True


In [5]:
# Berechnung der Anzahl der Fälle, in denen der vorherige Gewinner bzw. Verlierer das nächste Spiel gewinnt
prev_winner_wins = player_pairs_data['prev_winner_won_again'].sum()
prev_loser_wins = len(player_pairs_data) - prev_winner_wins

# Erstellung eines DataFrame für die Daten
data = pd.DataFrame({
    'Ergebnis des nächsten Spiels': ['Vorheriger Gewinner gewinnt', 'Vorheriger Verlierer gewinnt'],
    'Anzahl der Spiele': [prev_winner_wins, prev_loser_wins]
})

# Normierung der Daten
total_games = data['Anzahl der Spiele'].sum()
data['Anzahl der Spiele'] = data['Anzahl der Spiele'] / total_games

# Erstellung des Balkendiagramms
fig = px.bar(data, x='Ergebnis des nächsten Spiels', y='Anzahl der Spiele', 
             title='Vergleich der Ergebnisse auf Basis des vorherigen Spiels (normiert)')

# Anzeigen des Diagramms
fig.show()

Das Balkendiagramm zeigt den Vergleich zwischen der Anzahl der Spiele, in denen der Gewinner des vorherigen Spiels auch das nächste Spiel gewinnt (links), und der Anzahl der Spiele, in denen der Verlierer des vorherigen Spiels das nächste Spiel gewinnt (rechts). Wie zu sehen ist, gibt es eine höhere Tendenz, dass der Gewinner des letzten Spiels auch das nächste Spiel gegen denselben Gegner gewinnt, aber es gibt immer noch eine signifikante Anzahl von Fällen, in denen der Verlierer des vorherigen Spiels das nächste Spiel gewinnt.